instantiate package

In [20]:
using Pkg
Pkg.activate("../")
Pkg.instantiate()
using TeneT
using TeneT_demo

  Activating project at `e:\1 - research\4.9 - AutoDiff\TeneT_demo`


# 2D classical Ising model
The ''energy'' of a configuration $\sigma$ is given by the Hamiltonian function:

$H(\sigma) = -\sum_{\langle i~j\rangle} J \sigma_i \sigma_j$  ($J>0$)

The Boltzmann factor is:

$W_{i~j} = e^{-\beta E_{i~j}}$

We can solve this model by TeneT.jl in follow steps:

## Step 1: define M tensor
 <img src="../picture/build_M_tensor.png" width = "30%" height = "30%" align=center />

In [9]:
using Test

@testset "Ising model_tensor" for Ni in [1,2], Nj in [1,2]
    model = Ising(Ni,Nj,0.5)
    @test size(model_tensor(model, Val(:bulk)))   == (2,2,2,2,Ni,Nj)
    @test size(model_tensor(model, Val(:mag) ))   == (2,2,2,2,Ni,Nj)
    @test size(model_tensor(model, Val(:energy))) == (2,2,2,2,Ni,Nj)
end

Test Summary:      | Pass  Total
Ising model_tensor |    3      3
Test Summary:      | Pass  Total
Ising model_tensor |    3      3
Test Summary:      | Pass  Total
Ising model_tensor |    3      3
Test Summary:      | Pass  Total
Ising model_tensor |    3      3


4-element Vector{Any}:
 Test.DefaultTestSet("Ising model_tensor", Any[], 3, false, false)
 Test.DefaultTestSet("Ising model_tensor", Any[], 3, false, false)
 Test.DefaultTestSet("Ising model_tensor", Any[], 3, false, false)
 Test.DefaultTestSet("Ising model_tensor", Any[], 3, false, false)

## Step 2: use TeneT.obs_env to get environment

In [10]:
using Random
Random.seed!(100)

β = 0.5
model = Ising(1, 1, β)
M = model_tensor(model, Val(:bulk))
env = TeneT.obs_env(M; χ=10, maxiter=10, miniter=1, 
                    updown=false, verbose=true, show_every=1);

↑ 

random initial 1×1 vumps_χ10 environment-> 

vumps@step: 1, error=Inf


vumps@step: 2, error=0.01730159744333545
vumps@step: 3, error=0.0006110217932778949
vumps@step: 4, error=8.525419213658111e-7
vumps@step: 5, error=1.486693561895049e-7
vumps@step: 6, error=3.5210780596961775e-8
vumps@step: 7, error=7.535505333349743e-9


vumps@step: 8, error=1.640536608963289e-9
vumps@step: 9, error=3.7020363904278236e-10
vumps done@step: 9, error=8.508814802635002e-11

## Step 3: use env to calculate observable

In [11]:
using TeneT_demo: magofβ

@testset "$(Ni)x$(Nj) ising forward with $atype" for Ni = [1], Nj = [1], atype = [Array]
    Random.seed!(100)
    β = 0.5
    model = Ising(Ni, Nj, β)
    M = atype(model_tensor(model, Val(:bulk)))
    env = obs_env(M; χ = 10, maxiter = 10, miniter = 1, 
         infolder = "./example/data/$model/", 
        outfolder = "./example/data/$model/", 
        updown = false, verbose = false, savefile = false
        )
    @test observable(env, model, Val(:Z)     ) ≈ 2.789305993957602
    @test observable(env, model, Val(:mag)   ) ≈ magofβ(model) 
    @test observable(env, model, Val(:energy)) ≈ -1.745564581767667
end

Test Summary:                | Pass  Total
1x1 ising forward with Array |    3      3


1-element Vector{Any}:
 Test.DefaultTestSet("1x1 ising forward with Array", Any[], 3, false, false)

## Step 4(optional): calculate energy by Zygote.gradient

In [12]:
using Zygote

@testset "$(Ni)x$(Nj) ising backward with $atype" for Ni = [1], Nj = [1], atype = [Array]
    Random.seed!(100)
    function logZ(β)
        model = Ising(1, 1, β)
        M = model_tensor(model, Val(:bulk))
        env = obs_env(M;χ = 10, maxiter = 10, miniter = 1, 
                        updown = false, verbose = false, savefile = false
                    )
        log(real(observable(env, model, Val(:Z))))
    end
    @test Zygote.gradient(β->-logZ(β), 0.5)[1] ≈ -1.745564581767667
end

Test Summary:                 | Pass  Total
1x1 ising backward with Array |    1      1


1-element Vector{Any}:
 Test.DefaultTestSet("1x1 ising backward with Array", Any[], 1, false, false)

<font size=5>*try different Temperature $\beta$, bond dimension $\chi$, unit cell size $Ni \times Nj$ and CuArray with GPU!*</font>

# Finding the Ground State of infinite 2D Heisenberg model
The Heisenberg Hamiltonian function is:

$H = \sum_{\langle i~j\rangle} J^x S_i^x S_j^x + J^y S_i^y S_j^y + J^z S_i^z S_j^z $




In [13]:
@testset "hamiltonian" begin
    h = hamiltonian(Heisenberg(1,1))
    @test size(h) == (2,2,2,2)
    rh = reshape(permutedims(h,(1,3,2,4)),4,4)
    @test rh' == rh
end

Test Summary: | Pass  Total
hamiltonian   |    2      2


Test.DefaultTestSet("hamiltonian", Any[], 2, false, false)

The algorithm variationally minimizes the energy of a Heisenberg model on a two-dimensional infinite lattice using a form of gradient descent.

## Step 1: initial iPEPS tensor

In [14]:
@testset "init_ipeps" for Ni = [1,2], Nj = [1,2], D in [2,3], χ in [10]
    model = Heisenberg(Ni,Nj)
    A, key = init_ipeps(model; Ni=Ni, Nj=Nj, D=D, χ=χ);
    @test size(A) == (D,D,D,D,2,Ni,Nj)
end

random initial BCiPEPS ./data/1x1/Heisenberg{Float64}(1, 1, 1.0, 1.0, 1.0)\D2_χ10_tol1.0e-10_maxiter10.jld2


Test Summary: | Pass  Total
init_ipeps    |    1      1
random initial BCiPEPS ./data/1x1/Heisenberg{Float64}(1, 1, 1.0, 1.0, 1.0)\D3_χ10_tol1.0e-10_maxiter10.jld2
Test Summary: | Pass  Total
init_ipeps    |    1      1
random initial BCiPEPS ./data/1x2/Heisenberg{Float64}(1, 2, 1.0, 1.0, 1.0)\D2_χ10_tol1.0e-10_maxiter10.jld2
Test Summary: | Pass  Total
init_ipeps    |    1      1
random initial BCiPEPS ./data/1x2/Heisenberg{Float64}(1, 2, 1.0, 1.0, 1.0)\D3_χ10_tol1.0e-10_maxiter10.jld2
Test Summary: | Pass  Total
init_ipeps    |    1      1
random initial BCiPEPS ./data/2x1/Heisenberg{Float64}(2, 1, 1.0, 1.0, 1.0)\D2_χ10_tol1.0e-10_maxiter10.jld2
Test Summary: | Pass  Total
init_ipeps    |    1      1
random initial BCiPEPS ./data/2x1/Heisenberg{Float64}(2, 1, 1.0, 1.0, 1.0)\D3_χ10_tol1.0e-10_maxiter10.jld2
Test Summary: | Pass  Total
init_ipeps    |    1      1
random initial BCiPEPS ./data/2x2/Heisenberg{Float64}(2, 2, 1.0, 1.0, 1.0)\D2_χ10_tol1.0e-10_maxiter10.jld2
Test Summary: | 

8-element Vector{Any}:
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)
 Test.DefaultTestSet("init_ipeps", Any[], 1, false, false)

## Step 2: contract the tensor network to get the energy

We don't have constraints on ipeps, so the horizontal and vertical bond energy is different and we should use both the up and down environment.

In [15]:
using TeneT_demo: optcont, energy

@testset "energy" for Ni = [1], Nj = [1], D in [2,3], χ in [10]
    model = Heisenberg(Ni,Nj)
    A, key = init_ipeps(model; Ni=Ni, Nj=Nj, D=D, χ=χ)
    oc = optcont(D, χ)
    h = hamiltonian(model)
    @show energy(h, A, oc, key; verbose = true, savefile = true)
end

random initial BCiPEPS ./data/1x1/Heisenberg{Float64}(1, 1, 1.0, 1.0, 1.0)\D2_χ10_tol1.0e-10_maxiter10.jld2


Horizontal Contraction Complexity(seed=60)(18.501837184902293, 12.643856189774723)


Vertical Contraction Complexity(seed=60)(18.501837184902293, 12.643856189774723)


↑ random initial 1×1 vumps_χ10 environment-> 

vumps done@step: 6, error=2.7782652390435756e-11


...


Vertical energy = 0.24991961018128714 + 5.351124086022837e-12im
e = 0.4998353376157848 + 7.262177737050789e-12im
energy(h, A, oc, key; verbose = true, savefile = true) = 0.4998353376157848 + 7.262177737050789e-12im
Test Summary: |
energy        | No tests
random initial BCiPEPS ./data/1x1/Heisenberg{Float64}(1, 1, 1.0, 1.0, 1.0)\D3_χ10_tol1.0e-10_maxiter10.jld2
Horizontal Contraction Complexity(seed=60)(22.5340254495046, 14.983706192659348)
Vertical Contraction Complexity(seed=60)(22.5340254495046, 14.983706192659348)


↑ random initial 1×1 vumps_χ10 environment-> 

vumps done@step: 10, error=1.3757287350638443e-7
↓ random initial 1×1 vumps_χ10 environment-> 

vumps done@step: 10, error=5.428584177352408e-9
===========1,1===========
Horizontal energy = 0.24997823917856152 - 5.4618769783622356e-12im
Vertical energy = 0.2499600232174339 - 2.2576762081804176e-9im
e = 0.4999382623959954 - 2.26313808515878e-9im
energy(h, A, oc, key; verbose = true, savefile = true) = 0.4999382623959954 - 2.26313808515878e-9im
Test Summary: |
energy        | No tests


2-element Vector{Any}:
 Test.DefaultTestSet("energy", Any[], 0, false, false)
 Test.DefaultTestSet("energy", Any[], 0, false, false)

## Step 3: optimise the ipeps by gradient

The ground state of Heisenberg model is antiferromagnetic, which means we should use $2 \times 2$ unit cell to simulate it directly.

 <img src="../picture/antiferromagnetic_configuration.png" width = "30%" height = "30%" align=center />

In [21]:
using Optim
using Random

@testset "optimise_ipeps" for Ni = [2], Nj = [2], D in [2], χ in [10]
    Random.seed!(100)
    model = Heisenberg(Ni,Nj,1.0,1.0,1.0)
    A, key = init_ipeps(model; Ni=Ni, Nj=Nj, D=D, χ=χ, verbose= false)
    optimise_ipeps(A, key; f_tol = 1e-6, opiter = 20, optimmethod = LBFGS(m = 20))
end

Horizontal Contraction Complexity(seed=60)(18.501837184902293, 12.643856189774723)
Vertical Contraction Complexity(seed=60)(18.501837184902293, 12.643856189774723)
time  steps   energy           grad_norm


0.0   0       0.4762782727    0.1319273161


3.5   1       0.0754468132    0.16519771


5.0   2       -0.1938688851    0.0586885185


7.4   3       -0.5002361671    0.0445462218


...

73.9   18       -0.6600553975    0.0011398697


77.3   19       -0.6601239066    0.0007862693


81.0   20       -0.660150505    0.0004201797


Test Summary:  |
optimise_ipeps | No tests


1-element Vector{Any}:
 Test.DefaultTestSet("optimise_ipeps", Any[], 0, false, false)

<font size=5>*The configuration is ABBA, so we can simplify the ipeps by only using A and B two tensors. Try to verify it!*</font>

<font size=5>*If we only want ground state, the follow $1 \times 1$ unit cell is also correct. Think about why.*</font>

 Hint: Flip the second spin.

In [23]:
@testset "optimise_ipeps" for Ni = [1], Nj = [1], D in [2], χ in [10]
    Random.seed!(100)
    model = Heisenberg(Ni,Nj,-1.0,-1.0,1.0)
    A, key = init_ipeps(model; Ni=Ni, Nj=Nj, D=D, χ=χ, verbose= false)
    optimise_ipeps(A, key; f_tol = 1e-6, opiter = 20, optimmethod = LBFGS(m = 20))
end

Horizontal Contraction Complexity(seed=60)(18.501837184902293, 12.643856189774723)
Vertical Contraction Complexity(seed=60)(18.501837184902293, 12.643856189774723)
time  steps   energy           grad_norm


0.0   0       -0.4753796697    0.2117118078


4.3   1       -0.4881967138    0.1977033078


9.2   2       -0.5123718788    0.0803554653


16.4   3       -0.6069252459    0.1714754326


...

50.2   18       -0.6609888647    0.0040790792


52.6   19       -0.6611696515    0.0045124839


55.6   20       -0.6614515083    0.0045678184


Test Summary:  |
optimise_ipeps | No tests


1-element Vector{Any}:
 Test.DefaultTestSet("optimise_ipeps", Any[], 0, false, false)